In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import Counter

def scrape_reviews(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        avg_rating = soup.find('p', class_='ui-review-capability__rating__average').text.strip()
        num_reviews = soup.find('p', class_='ui-review-capability__rating__label').text.strip()

        reviews = []
        review_elements = soup.find_all('article', class_='ui-review-capability-comments__comment')

        for element in review_elements:
            rating_text = element.find('p', class_='andes-visually-hidden').text.strip()
            review_text = element.find('p', class_='ui-review-capability-comments__comment__content').text.strip()
            review_date = element.find('span', class_='ui-review-capability-comments__comment__date').text.strip()
            
            rating_value = int(rating_text.split()[1])
            sentiment = 'Positiva' if rating_value >= 4 else 'Negativa'

            reviews.append({
                'rating': rating_value,
                'review': review_text,
                'date': review_date,
                'sentiment': sentiment
            })

        rating_counts = Counter([review['rating'] for review in reviews])
        positive_reviews = sum(1 for review in reviews if review['sentiment'] == 'Positiva')
        positive_percentage = (positive_reviews / len(reviews)) * 100 if reviews else 0

        return {
            'avg_rating': avg_rating,
            'num_reviews': num_reviews,
            'rating_counts': rating_counts,
            'positive_percentage': positive_percentage,
            'reviews': reviews
        }

    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return None

def scrape_product_info(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h1', class_='ui-pdp-title').text.strip()
        price = soup.find('span', class_='andes-money-amount__fraction').text.strip()
        description_tag = soup.find('p', class_='ui-pdp-description__content')
        description = description_tag.text.strip() if description_tag else "No disponible"

        return {
            'title': title,
            'price': price,
            'description': description
        }

    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return None
    except AttributeError as e:
        print("Error al extraer información del producto:", e)
        return None

# URL de ejemplo
url = "https://www.mercadolibre.com.co/impresora-simple-funcion-a-color-epson-ecotank-l121-110v/p/MCO24554271?pdp_filters=deal%3AMCO779366-1"

# Obtener la información del producto
product_info = scrape_product_info(url)

# Obtener las reseñas
reviews_data = scrape_reviews(url)

# Mostrar los resultados
if product_info:
    print(f"Título: {product_info['title']}")
    print(f"Precio: {product_info['price']}")
    print(f"Descripción: {product_info['description']}")

if reviews_data:
    print(f"\nCalificación promedio: {reviews_data['avg_rating']}")
    print(f"Número de calificaciones: {reviews_data['num_reviews']}")
    print(f"Distribución de calificaciones: {reviews_data['rating_counts']}")
    print(f"Porcentaje de opiniones positivas: {reviews_data['positive_percentage']:.2f}%")

    print("\nOpiniones destacadas:")
    for review in reviews_data['reviews']:
        print(f"Fecha: {review['date']}")
        print(f"Calificación: {review['rating']}")
        print(f"Sentimiento: {review['sentiment']}")
        print(f"Reseña: {review['review']}\n")


Calificación promedio: 4.9
Número de calificaciones: 35 calificaciones
Distribución de calificaciones: Counter({5: 2})
Porcentaje de opiniones positivas: 100.00%

Opiniones destacadas:
Fecha: 16 ene. 2024
Calificación: 5
Sentimiento: Positiva
Reseña: Un excelente producto cuando lo único que necesitas es imprimir y no requieres las demás funciones de una multifuncional.

Fecha: 24 mar. 2024
Calificación: 5
Sentimiento: Positiva
Reseña: Llega súper rápido, impresora muy compacta para espacios pequeños, tal cual la buscaba.

